<a href="https://colab.research.google.com/github/989077/ML_internship_program_codetech_sol/blob/main/recomadation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Recommendation System : collaborative system + Matrix Factorization

In [ ]:
from zipfile import ZipFile, ZipExtFile # Import ZipFile class explicitly
file_name = '/content/movie_recc.zip'
with ZipFile('/content/movie_recc.zip','r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('/content/movie recommendation system/movies.csv')
ratings_df = pd.read_csv('/content/movie recommendation system/ratings.csv')


In [ ]:
movies_df.head(100)


,movie_id,title,genre,release_year
0,1,Rest.,Horror,1986
1,2,Sport admit.,Sci-Fi,1987
2,3,Term reality.,Horror,1981
3,4,Government your only.,Thriller,2020
4,5,Her pretty north.,Mystery,1983
...,...,...,...,...
95,96,Old fall on.,Drama,1999
96,97,Range huge of.,Romance,2008
97,98,Suggest point nearly.,Fantasy,1996
98,99,Thank sell.,Drama,2003


In [ ]:
ratings_df.head()


,user_id,movie_id,rating
0,18,76,3
1,6,92,1
2,7,53,5
3,6,86,1
4,34,30,1


In [ ]:
# movie ID to movie mapping
movie_name = movies_df.set_index('movie_id')['title'].to_dict()
n_users = len(ratings_df.user_id.unique())
n_movies = len(ratings_df.movie_id.unique())

In [ ]:
print(n_users)
print(n_movies)

50
99


In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
  def __init__(self,n_users,n_movies,n_factors=20):
    super().__init__()
    # create  user embedding
    self.user_factors = torch.nn.Embedding(n_users,n_factors)
    # Create items embedding
    self.item_factor = torch.nn.Embedding(n_movies,n_factors)
    self.user_factors.weight.data.uniform_(0,0.05)
    self.item_factor.weight.data.uniform_(0,0.05)
  def forward(self,data):
    # matrix multiplication
    users,items = data[:,0],data[:,1]
    return (self.user_factors(users)*self.item_factor(items)).sum(1)
    #
  def predict(self,user,item):
    return self.forward(user,item)





In [ ]:
class loader(Dataset):
  def __init__(self):
    self.ratings = ratings_df.copy()
    # Exract all user IDs and movie IDs
    users = ratings_df.user_id.unique()
    movies = ratings_df.movie_id.unique()
    # --creating new continuos
    self.userid2idx = {o:i for i,o in enumerate(users)}
    self.movieid2idx = {o:i for i,o in enumerate(movies)}
    # return the id from the indexed value as noted inthe labda function down below.
    self.ratings.movie_id = ratings_df.movie_id.apply(lambda x: self.movieid2idx[x])
    self.ratings.user_id = ratings_df.user_id.apply(lambda x: self.userid2idx[x])

    self.y = self.ratings['rating'].values
    # Create self.x using user_id and movie_id columns
    self.x = self.ratings[['user_id', 'movie_id']].values
    self.x,self.y = torch.tensor(self.x),torch.tensor(self.y) # tranform the data to tensor

  def __getitem__(self, index):
    return (self.x[index],self.y[index])
  def __len__(self):
    return len(self.ratings)

In [ ]:
num_epochs = 128
cuda =torch.cuda.is_available()
print("Is running on the GPU :",cuda)

model = MatrixFactorization(n_users,n_movies,20)
print(model)

for name,parameters in model.named_parameters():
  if parameters.requires_grad:
    print(name, parameters.data)
# GPU is  enable if you have a GPU
if cuda:
  model = model.cuda()
# MSE loss
loss_fn = torch.nn.MSELoss()
# adam optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

# train data
train_set = loader()
train_loader = DataLoader(train_set,128,shuffle=True)

Is running on the GPU : True
MatrixFactorization(
  (user_factors): Embedding(50, 20)
  (item_factor): Embedding(99, 20)
)
user_factors.weight tensor([[2.8951e-02, 4.9524e-02, 2.7662e-02, 4.2588e-02, 2.5221e-02, 3.5816e-02,
         1.0121e-02, 6.1283e-03, 2.0002e-02, 1.7852e-02, 4.9088e-02, 2.8317e-02,
         4.0114e-02, 2.7768e-02, 1.9511e-02, 4.2492e-02, 1.7235e-02, 4.2330e-02,
         4.8452e-03, 3.2426e-02],
        [1.8217e-02, 1.7088e-02, 2.4547e-03, 3.8195e-02, 1.7715e-02, 2.8929e-03,
         2.7387e-03, 3.1218e-02, 4.8363e-02, 2.1595e-02, 2.2620e-02, 4.4876e-02,
         1.9803e-02, 4.6901e-02, 3.9236e-02, 2.0219e-02, 3.0731e-02, 2.5674e-02,
         9.6434e-04, 3.0606e-02],
        [2.0207e-02, 3.1928e-02, 4.5598e-02, 3.0349e-03, 2.4879e-02, 3.4119e-02,
         3.8548e-02, 3.4070e-02, 4.3195e-02, 8.4305e-03, 4.9245e-02, 2.8170e-02,
         4.4679e-02, 3.9239e-02, 3.9678e-02, 3.0115e-03, 3.4768e-02, 1.0240e-02,
         3.9751e-02, 3.4428e-02],
        [6.3554e-03, 3.708

In [ ]:
for it in range(num_epochs):
  losses = []
  for x,y in tqdm(train_loader):
    if cuda:
      x,y = x.cuda(),y.cuda()
      optimizer.zero_grad()
      outputs = model(x)
      loss = loss_fn(outputs.squeeze(),y.type(torch.float32))
      losses.append(loss.item())
      loss.backward()
      optimizer.step()
    print('iter a{}'.format(it), 'Loss', sum(losses) / len(losses))

<ipython-input-50-1ce4c60ff48b>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x,y in tqdm(train_loader):


  0%|          | 0/4 [00:00<?, ?it/s]

iter a0 Loss 11.810906410217285
iter a0 Loss 11.008626461029053
iter a0 Loss 10.835704485575357
iter a0 Loss 10.968363523483276


  0%|          | 0/4 [00:00<?, ?it/s]

iter a1 Loss 10.397197723388672
iter a1 Loss 10.705972671508789
iter a1 Loss 10.917736371358236
iter a1 Loss 10.938381910324097


...

iter a125 Loss 1.3980095386505127
iter a125 Loss 1.3705546855926514
iter a125 Loss 1.3725037972132366
iter a125 Loss 1.4390054643154144


  0%|          | 0/4 [00:00<?, ?it/s]

iter a126 Loss 1.443779468536377
iter a126 Loss 1.5025413036346436
iter a126 Loss 1.4567745526631672
iter a126 Loss 1.4276234209537506


  0%|          | 0/4 [00:00<?, ?it/s]

iter a127 Loss 1.2728478908538818
iter a127 Loss 1.3351507186889648
iter a127 Loss 1.4005502462387085
iter a127 Loss 1.4276388883590698


In [ ]:
# by training model, we will have tuned latest factors for movies and users
c = 0
Uw = 0
iw = 0
for name, param in model.named_parameters():
  if param.requires_grad:
    print(name, param.data)
    if c == 0:
      Uw = param.data
      c += 1
    else:
      iw = param.data


user_factors.weight tensor([[0.3919, 0.4212, 0.4046, 0.4018, 0.3950, 0.4021, 0.3780, 0.3672, 0.3937,
         0.3884, 0.4132, 0.3987, 0.4123, 0.3969, 0.3848, 0.4163, 0.3935, 0.4069,
         0.3674, 0.4022],
        [0.3875, 0.3836, 0.3731, 0.4044, 0.3842, 0.3752, 0.3739, 0.4069, 0.4256,
         0.3883, 0.4000, 0.4077, 0.3831, 0.4080, 0.4043, 0.3870, 0.3984, 0.3991,
         0.3649, 0.3973],
        [0.3947, 0.4043, 0.4238, 0.3828, 0.3993, 0.4082, 0.4095, 0.4025, 0.4168,
         0.3863, 0.4275, 0.4061, 0.4181, 0.4201, 0.4183, 0.3894, 0.4140, 0.3994,
         0.4252, 0.4208],
        [0.3878, 0.4191, 0.3937, 0.4132, 0.3958, 0.4094, 0.4063, 0.4241, 0.3996,
         0.4256, 0.3982, 0.4082, 0.3991, 0.3918, 0.4228, 0.4217, 0.3888, 0.4030,
         0.3943, 0.4212],
        [0.4101, 0.4447, 0.4388, 0.4031, 0.4305, 0.4195, 0.4485, 0.4434, 0.3949,
         0.3914, 0.4116, 0.4028, 0.4036, 0.4097, 0.3973, 0.4084, 0.4162, 0.4098,
         0.4277, 0.4344],
        [0.4648, 0.4395, 0.4773, 0.4657,

In [ ]:
 trained_movie_embeddings =model.item_factor.weight.data.cpu().numpy()
 trained_movie_embeddings

array([[0.39997664, 0.37021354, 0.40630418, ..., 0.39675385, 0.4176277 ,
        0.3882386 ],
       [0.28921956, 0.27705383, 0.2911969 , ..., 0.3046862 , 0.29520574,
        0.265044  ],
       [0.46767032, 0.45454726, 0.46968967, ..., 0.44280007, 0.4457132 ,
        0.44115505],
       ...,
       [0.23088908, 0.23541117, 0.20865324, ..., 0.22955117, 0.25542623,
        0.2578473 ],
       [0.4119672 , 0.42435735, 0.41682988, ..., 0.40933344, 0.42937678,
        0.3937972 ],
       [0.11405914, 0.09679921, 0.11303463, ..., 0.11474976, 0.13357581,
        0.11982543]], dtype=float32)

In [ ]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [ ]:
# Creating clusters
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        if movidx not in train_set.movieid2idx:
            continue  # Skip invalid movidx
        movid = train_set.movieid2idx[movidx]
        if movid not in ratings_df['movie_id'].values:
            continue  # Skip if movid is not in ratings_df
        rat_count = ratings_df.loc[ratings_df['movie_id'] == movid, 'user_id'].count()
        movs.append((movie_name[movid], rat_count))
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", mov[0], mov[1])  # Use the count from the tuple


Cluster #0
	 Well nearly baby. 8
	 Expect stuff become right. 8
	 Fire cold head other. 7
	 Oil office detail. 7
	 Modern present. 6
	 Building throughout among. 6
	 Language financial. 5
	 Key know. 4
	 Black generation. 3
	 Specific reduce. 3
Cluster #1
	 Degree upon. 8
	 Tonight short pressure. 6
	 White. 6
	 Audience little. 5
	 Against along. 5
	 Whether simply eat. 5
	 A effect. 4
	 Nothing organization. 3
Cluster #2
	 Single wear news maybe. 10
	 Can simple. 7
	 Financial treat. 6
	 Pull personal. 6
	 Class answer continue think. 6
	 Growth financial sort. 5
	 Clearly skill himself each. 4
	 Mr maybe remain. 2
Cluster #3
	 Sign police how. 10
	 Local reality make. 9
	 Surface wall any. 7
	 Grow building hour. 7
	 Indicate happen. 6
	 Business discuss. 4
	 Off service. 4
	 Quite defense present. 4
	 Rest. 4
	 Bad. 3
Cluster #4
	 Her pretty north. 5
	 Suggest point nearly. 2
Cluster #5
	 Billion there her. 9
	 More daughter town. 7
	 Series play college. 6
	 Task movement. 6
	 Fig

ValueError: Expected 2D array, got scalar array instead:
array=91.0.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.